In [ ]:
using Gridap
using Gridap
using GridapMakie, CairoMakie, FileIO
using Gridap.FESpaces
using Gridap.ReferenceFEs
using Gridap.Arrays
using Gridap.Algebra
using Gridap.Geometry
using Gridap.Fields
using Gridap.CellData
using FillArrays
using Test
using InteractiveUtils

function GradientDescent(;solveSE, solveAE, spaces, dΩ, dΓ=nothing, Q, J, ∇f, iter_max=1000, tol=1e-3, Proj=x->x, u0=nothing, w=nothing, s_min=nothing, sminargs=nothing, armijoparas=(ρ=1/2, α_0=1, α_min=1/2^5, σ=1e-4), Δt=0.05, t0=0.0, tF, saveall::Bool=false)
	Trialspace, Testspace, Qspace = spaces                                  # Extract spaces

	q = [(t,interpolate_everywhere(Q(t),Qspace(t))) for t=t0:Δt:tF]   
	qfun(t)=find(q,t)
	println("q computed")

	y, cacheSE, A_SE = solveSE(Q,Trialspace,Testspace,w=w,dΩ=dΩ,dΓ=dΓ)  # initial SE solve
	#y = FEFunction(Trialspace, y_dof)
	yfun(t) = find(y,t)

	p, cacheAE, A_AE = solveAE(yfun,q,Trialspace,Testspace,dΩ=dΩ,dΓ=dΓ)      # initial AE solve
	#p = FEFunction(Testspace, p_dof)
	println("p computed")

	cost = J(y, q)
	fgrad =  ∇f(q, p, y)														# Compute initial gradient
	L2fgrad_save = L2norm(fgrad)                                       # Compute norm of initial gradient

	if saveall
		qs=[q]                     # save the solutions - only if really necessary
		ys=[y]
		ps=[p]
		costs=[cost]
	else
		qs,ys,ps=[],[],[]
	end
	for k=1:iter_max
		println("entered for loop, E=$cost")
		q_new = y_new = cost_new = nothing
		if s_min!=nothing                                                   # if method for exact step size is defined use that
			s = s_min(y,q,w,p,sminargs=(cacheSE,A_SE,sminargs))
			q_new = interpolate_everywhere(q - s*fgrad,Qspace) |> P			# in most cases interpolate instead of interpolate_everywhere works as well
			y_dof, cacheSE = solveSE(q_new,Trialspace,Testspace;w=w,dΩ=dΩ,dΓ=dΓ)
			y_new = FEFunction(Trialspace, y_dof)
			cost_new = J(y,q)
		else                                                                # else use Armijo rule - if projection is used, you should consider smaller step size than default
			ρ, α_0, α_min, σ = armijoparas
			cost_new = cost
			L2fgrad = L2fgrad_save
			α = α_0
			while α > α_min
				println("entered while loop")
				
				
				q_new = [(t,interpolate_everywhere((qfun(t) - α*grad)*q_pos, Qspace(t))) for (t,grad) in fgrad]   # Compute tentative new control function defined by current line search parameter
				
				qfun=t->find(q_new,t)

				y_new, cacheSE = solveSE(qfun,Trialspace,Testspace;w=w,dΩ=dΩ,dΓ=dΓ)
				#y_new = FEFunction(Trialspace, y_dof)

				cost_new = J(y_new, q_new)                                  # Compare decrease in functional and accept if sufficient
				if cost_new < cost - σ*α*L2fgrad^2
					break
				else
                    # α = α_min #TEST!!
					α *= ρ
				end
			end
			if α <= α_min
				println("Armijo rule failed")
				break
			end

		end# here you could implement other methods
		println("Armoijo success")
		q = q_new
		y = y_new
		cost = cost_new
		
		if k % 10 == 0
			println("iteration: $k")
		end
		if saveall
			push!(qs,q)
			push!(ys,y)
			push!(costs,cost)
		end
		yfun=t->find(y,t)
		p, cacheAE = solveAE(yfun,q,Trialspace,Testspace;dΩ=dΩ,dΓ=dΓ)
		#p = FEFunction(Testspace, p_dof)

		fgrad = ∇f(q, p, y)
		L2fgrad = L2norm(fgrad)
		push!(ps,p)

		if L2fgrad < tol*L2fgrad_save                           # loop break condition - better ideas are appreciated
			break
		end

	end
	return saveall ? (ys,qs,ps,costs) : (y,q,p,cost)                    	# give back either all saved variables or only end result
end



function χ(x, a, b)
    if a < x && x < b
        return 1
    else
        return 0
    end
end

# Set parameters and run the solver
domain = (-1, +1, -1, +1)
partition = (20, 20)
order = 1
degree = 2
Δt = 0.01
θ = 0.5
solver = ThetaMethod(LUSolver(), Δt, θ)
t0, tF = 0.0, 1.0

model = CartesianDiscreteModel(domain, partition) |> simplexify #Koden fungerer ikke uten simplexify....????
reffe = ReferenceFE(lagrangian, Float64, order)
Testspace = TestFESpace(model, reffe, conformity=:H1)
Trialspace = TransientTrialFESpace(Testspace)

Ω = Triangulation(model)
dΩ = Measure(Ω, degree)
Γ = BoundaryTriangulation(model)
dΓ = Measure(Γ, degree)

Uspace = FESpace(model, reffe, conformity=:H1)



q_pos(x) = χ(x[1], -0.9, -0.7) * χ(x[2], 0, 0.20) + χ(x[1], 0.2, 0.4) * χ(x[2], -0.5, 0)
q(t) = x -> χ(t,0.0,1.0)*1000.0*q_pos(x)


Tout(t) = x -> 4.0



ρ(x)=1
c(x)=1
k(x)=1
h(x)=1
Toutdoor(x,t)=5.0
Tout(t)=x->Toutdoor(x,t)
Q(x,t)=1000.0*q_pos(x)*χ(t,0.0,1.0)
Qt(t)=x->Q(x,t)
price(t)=1.0
Tini(x)=20.0
t0=0.0
tF=1.0
TIni=interpolate_everywhere(Tini, Uspace(t0))
Tfin=interpolate_everywhere(20.0, Uspace(tF))
Δt = 0.05

γ = 1

function solveSE(Qt,Trialspace,Testspace;w=nothing,dΩ,dΓ=nothing,cache=nothing,A=nothing,y_dof=fill(0.0, num_free_dofs(Testspace)))
    
    α(t) = x -> 1.0
    m(t, dtu, v) = ∫(v * dtu)*dΩ
    a(t, u, v) = ∫(α(t) * ∇(v) ⋅ ∇(u))*dΩ + ∫(u * v)*dΓ
    l(t, v) = ∫(q_pos*v * Qt(t))* dΩ + ∫(Tout(t) * v)*dΓ
    
    SEop_opt = TransientLinearFEOperator((a, m), l, Trialspace, Testspace, constant_forms=(true, true))
    T0 = interpolate_everywhere(20.0, Trialspace(t0))
    T = solve(solver, SEop_opt, t0, tF, T0)

    return [(t0,T0),collect((t,TT) for (t,TT) in T)...], 0.0,0.0  

end

function solveAE(T,Q,Trialspace,Testspace;dΩ,dΓ=nothing,cache=nothing,A=nothing,W_dof=fill(0.0, num_free_dofs(Testspace)))
    α(t) = x -> 1.0
    m_AE(t, dtu, v) = ∫(v * dtu)dΩ
    a_AE(t, u, v) = ∫(α(t) * ∇(v) ⋅ ∇(u))dΩ + ∫(u*v)dΓ # +??
    l_AE(t, v) = ∫(0.0*v)dΩ

    AEop_opt = TransientLinearFEOperator((a_AE, m_AE), l_AE, Trialspace, Testspace, constant_forms=(true, true))

    W_end = interpolate_everywhere(T(tF)-20.0,Trialspace(tF))  
    W = solve(solver, AEop_opt, t0, tF, W_end)


	return [collect((tF-t,w) for (t,w) in reverse(collect((t,WW) for (t,WW) in W)))... ; (tF,W_end)], 0.0,0.0
end

function E(T,Q)
	E=0.0
	for (QQ,t) in Q
		E+=Δt*price(t)^2*γ*∑(∫(QQ*QQ)*dΩ)
	end
	tmp = last(Q[1])-20
	E += γ* ∑(∫(tmp*tmp)*dΩ)
	# for (TT,t) in T
	# 	tmp=TT-20.0
	# 	E+=Δt*∑(∫(tmp*tmp)*dΩ) #The whole time?
	# 	println("Once")
	# end
	E = E/2.0
	return E
end
a = 0
b = 1000
Proj(a,b,z) = min(max(a,z),b)
# Proj(z) = map(x->Proj(a,b,x),z)

Proj(z) = [(t,FEFunction(Uspace,map(x->Proj(a,b,x), get_free_dof_values(zz)))) for (t,zz) in z]
# [(t,interpolate_everywhere(Uspace,map(x->Proj(a,b,x), Uspace(t)))) for (t,zz) in z]


∇e(Q::Vector{Tuple{Float64, SingleFieldFEFunction{GenericCellField{ReferenceDomain}}}},
T::Vector{Tuple{Float64, SingleFieldFEFunction{GenericCellField{ReferenceDomain}}}},
W::Vector{Tuple{Float64, SingleFieldFEFunction{GenericCellField{ReferenceDomain}}}}) = [(Q[k][1],2*price(T[k][1])^2*Q[k][2]-W[k][2]) for k=1:length(Q)]                                                # gradient of reduced cost
function ∇e(Qt::Function,
T::Vector{Tuple{Float64, SingleFieldFEFunction{GenericCellField{ReferenceDomain}}}},
W::Vector{Tuple{Float64, SingleFieldFEFunction{GenericCellField{ReferenceDomain}}}})
	println(T)
	println(W)
	return collect((T[k][1],interpolate_everywhere(2*price(T[k][1])^2*Qt(T[k][1])-W[k][2],Uspace(T[k][1]))) for k=1:length(W))
end

function find(y,s)
	tsave,ysave=t0,nothing
	for (t,yy) in y
		ysave=yy
		break
	end
	for (t,yy) in y
		# if t≈t0
		# 	ysave=yy
		# 	tsave=t0
		# end
		if t≈s
			return yy
		elseif t ≥ s
			return interpolate_everywhere(((s-tsave)*ysave+(t-s)*yy)/Δt, Uspace(s))
		end
		tsave,ysave = t,yy
	end
end


function s_min(y,q,w,p;sminargs)
	cache,A=sminargs
	v=interpolate(-y*p+γ*q, Uspace)                                      # v=p+γ*u, since all uses of v are in norms we can ignore the minus
	Sv,_ = SEsolver(v,Trialspace,Testspace;dΩ=dΩ,cache=cache,A=A)
	Sv = FEFunction(Trialspace,Sv)
	L2NormSquaredOfv = ∑(∫(v*v)*dΩ)                                   # ||v||^2
	return L2NormSquaredOfv/(∑(∫(Sv*Sv)*dΩ)+γ*L2NormSquaredOfv)
end

L2norm(u)=√((tF-t0)*∑(Δt*∑(∫(uu⋅uu)*dΩ) for (t,uu) in u))

(ys,qs,ps,costs) = GradientDescent(;solveSE=solveSE, 
solveAE=solveAE, 
spaces=(Trialspace, Testspace, Uspace), 
dΩ=dΩ, 
dΓ=dΓ, 
Q=Qt, 
J=E, 
∇f=∇e, 
Proj=Proj, 
s_min=nothing,
sminargs=nothing, 
saveall=true, 
tol=1e-5, 
iter_max=4, 
armijoparas=(ρ=1/2, α_0=1, α_min=1/2^20, σ=1e-4), 
Δt=Δt, 
t0=t0, 
tF=tF)
